In [1]:
import joblib

# Load the saved models
rf_model = joblib.load('rf_model.pkl')
lgb_model = joblib.load('lgb_model.pkl')


In [5]:
val_data=pd.read_csv('validation_data.csv')

In [6]:
df=pd.DataFrame(val_data)

In [7]:
missing_percent = df.isnull().sum() / len(df) * 100
print(missing_percent.sort_values(ascending=False))

bureau_436           100.000000
bureau_447           100.000000
bureau_449            93.989280
bureau_148            93.362366
bureau_448            89.995693
                        ...    
onus_attribute_37      0.000000
onus_attribute_38      0.000000
onus_attribute_39      0.000000
onus_attribute_40      0.000000
account_number         0.000000
Length: 1215, dtype: float64


In [8]:
df = df.dropna(axis=1, how='all')
columns_to_drop = df.columns[df.isna().all()]
print(columns_to_drop)

Index([], dtype='object')


In [9]:
# Drop columns with more than 70% missing values
threshold = 0.7 * len(df)
df = df.dropna(axis=1, thresh=threshold)



In [10]:
from sklearn.impute import SimpleImputer

# Median imputation for numerical columns
imputer = SimpleImputer(strategy='median')
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = imputer.fit_transform(df[num_cols])


In [12]:
import pickle

In [16]:
# Load the pickled pipeline
with open('rf_model.pkl', 'rb') as file:
    pipeline = pickle.load(file)

# Get the list of expected feature names
expected_columns = pipeline.named_steps['imputer'].feature_names_in_


In [28]:
# Load the pickled pipeline
with open('lgb_model.pkl', 'rb') as file:
    lgpipeline = pickle.load(file)

# Get the list of expected feature names
expected_columns = pipeline.named_steps['imputer'].feature_names_in_


In [17]:
# Ensure X_val matches the expected columns
X_val = val_data[expected_columns]


In [29]:
# Ensure X_val matches the expected columns
X_valgb = val_data[expected_columns]


In [18]:
# Add missing columns (if any) with default values
for col in expected_columns:
    if col not in X_val.columns:
        X_val[col] = 0  # Or np.nan, depending on your imputer

# Reorder columns to match expected order
X_val = X_val[expected_columns]


In [30]:
# Add missing columns (if any) with default values
for col in expected_columns:
    if col not in X_val.columns:
        X_valgb[col] = 0  # Or np.nan, depending on your imputer

# Reorder columns to match expected order
X_valgb = X_valgb[expected_columns]


In [20]:
# Load pickled pipeline
with open('rf_model.pkl', 'rb') as file:
    pipeline = pickle.load(file)

# Ensure validation data matches the training data structure
expected_columns = pipeline.named_steps['imputer'].feature_names_in_
X_val = val_data[expected_columns]  # Select and reorder columns
for col in expected_columns:
    if col not in X_val.columns:
        X_val[col] = 0  # Add missing columns

# Predict probabilities
y_pred_prob = pipeline.predict_proba(X_val)[:, 1]

# Save predictions
predictions = pd.DataFrame({
    'account_number': val_data['account_number'],  # Include account_number if available
    'predicted_prob': y_pred_prob
})
predictions.to_csv('predictions.csv', index=False)


In [34]:
# Load pickled pipeline
with open('lgb_model.pkl', 'rb') as file:
    lgpipeline = pickle.load(file)

# Ensure validation data matches the training data structure
expected_columns = pipeline.named_steps['imputer'].feature_names_in_
X_valgb = val_data[expected_columns]  # Select and reorder columns
for col in expected_columns:
    if col not in X_valgb.columns:
        X_valgb[col] = 0  # Add missing columns

# Predict probabilities
y_pred_problg = lgpipeline.predict_proba(X_valgb)[:, 1]

# Save predictions
predictions = pd.DataFrame({
    'account_number': val_data['account_number'],  # Include account_number if available
    'predicted_prob': y_pred_problg
})
predictions.to_csv('lgbpredictions.csv', index=False)


/Users/ishamadlani/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [21]:
# Predict probabilities
y_pred_prob = pipeline.predict_proba(X_val)[:, 1]

# Convert probabilities to class predictions
y_pred_class = (y_pred_prob >= 0.5).astype(int)  # Threshold of 0.5 for classification

# Count predictions for each class
count_zeros = (y_pred_class == 0).sum()
count_ones = (y_pred_class == 1).sum()

print(f"Number of predictions for class 0: {count_zeros}")
print(f"Number of predictions for class 1: {count_ones}")


Number of predictions for class 0: 41600
Number of predictions for class 1: 192


In [36]:
# Predict probabilities
y_pred_problg = pipeline.predict_proba(X_val)[:, 1]

# Convert probabilities to class predictions
y_pred_classlg = (y_pred_problg >= 0.5).astype(int)  # Threshold of 0.5 for classification

# Count predictions for each class
count_zeros = (y_pred_classlg == 0).sum()
count_ones = (y_pred_classlg == 1).sum()

print(f"Number of predictions in lgbm model for class 0: {count_zeros}")
print(f"Number of predictions in lgbm model for class 1: {count_ones}")


/Users/ishamadlani/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Number of predictions in lgbm model for class 0: 41432
Number of predictions in lgbm model for class 1: 360


In [43]:
import pandas as pd

# Load predictions
predictions = pd.read_csv("predictions.csv", header=None, names=["account_number", "predicted_prob"])

# Convert the 'predicted_prob' column to numeric, forcing errors to NaN
predictions["predicted_prob"] = pd.to_numeric(predictions["predicted_prob"], errors='coerce')

# Check if there are any NaN values (non-numeric entries)
print(predictions[predictions["predicted_prob"].isna()])

# Optionally, drop rows with NaN in 'predicted_prob'
predictions = predictions.dropna(subset=["predicted_prob"])

# Convert probabilities to binary predictions
predictions["bad_flag"] = (predictions["predicted_prob"] >= 0.5).astype(int)

# Save updated predictions
predictions[["account_number", "bad_flag"]].to_csv("final_predictions.csv", index=False)


   account_number  predicted_prob
0  account_number             NaN


In [38]:
import pandas as pd

# Load predictions
predictions = pd.read_csv("lgbpredictions.csv", header=None, names=["account_number", "predicted_prob"])

# Convert the 'predicted_prob' column to numeric, forcing errors to NaN
predictions["predicted_prob"] = pd.to_numeric(predictions["predicted_prob"], errors='coerce')

# Check if there are any NaN values (non-numeric entries)
print(predictions[predictions["predicted_prob"].isna()])

# Optionally, drop rows with NaN in 'predicted_prob'
predictions = predictions.dropna(subset=["predicted_prob"])

# Convert probabilities to binary predictions
predictions["bad_flag"] = (predictions["predicted_prob"] >= 0.5).astype(int)

# Save updated predictions
predictions[["account_number", "bad_flag"]].to_csv("final_predictionslg.csv", index=False)


   account_number  predicted_prob
0  account_number             NaN


In [44]:
import pandas as pd

# Read the final_predictions.csv file
predictions = pd.read_csv("rf_final_predictions.csv")

# Count the occurrences of 0 and 1 in the 'bad_flag' column
bad_flag_counts = predictions["bad_flag"].value_counts()

# Display the counts of 0 and 1
print("RANDOM FOREST MODEL PREDICTIONS")
print(bad_flag_counts)


RANDOM FOREST MODEL PREDICTIONS
bad_flag
0    41600
1      192
Name: count, dtype: int64


In [45]:
import pandas as pd

# Read the final_predictions.csv file
predictions = pd.read_csv("lgfinal_predictions.csv")

# Count the occurrences of 0 and 1 in the 'bad_flag' column
bad_flag_counts = predictions["bad_flag"].value_counts()

# Display the counts of 0 and 1
print("LGBM MODEL PREDICTIONS")
print(bad_flag_counts)


LGBM MODEL PREDICTIONS
bad_flag
0    41432
1      360
Name: count, dtype: int64
